### Bronze Layer Plan

In the Bronze layer, we ingest raw source files and convert them into **Delta format** for reliability, versioning, and schema evolution.

Due to the use of **Unity Catalog Volumes** for governance, Bronze data is stored as **Delta files inside a governed volume**, not as Unity Catalog tables.

#### Why we do NOT create Unity Catalog tables in Bronze
- The Bronze storage path is managed by a **Unity Catalog Volume**.
- Unity Catalog does not allow `CREATE TABLE … LOCATION` on paths that are already governed by a Volume.
- This design prevents overlapping governance and accidental permission bypass.
- As a result, Bronze data is accessed **by volume paths**, not by table names.


In [0]:
%sql
CREATE EXTERNAL VOLUME sales_databricks_workspace.bronze.bronze_retail_vol
LOCATION 'abfss://sales-data@saleslakestorage.dfs.core.windows.net/bronze';


In [0]:
from pyspark.sql.functions import current_timestamp, col

RAW_BASE = "/Volumes/sales_databricks_workspace/raw/lakehouse_retail_vol/sales/3a_superstore"
BRONZE_BASE = "/Volumes/sales_databricks_workspace/bronze/bronze_retail_vol/sales/3a_superstore"

def ingest_to_bronze(table_name, delimiter):
    raw_path = f"{RAW_BASE}/{table_name}"
    bronze_path = f"{BRONZE_BASE}/{table_name}"

    df = (
        spark.read
             .option("header", "true")
             .option("inferSchema", "true")
             .option("delimiter", delimiter)
             .csv(raw_path)
             .withColumn("ingestion_ts", current_timestamp())
             .withColumn("source_file", col("_metadata.file_path"))
    )

    df.write.format("delta") \
      .mode("overwrite") \
      .option("mergeSchema","true")\
      .save(bronze_path)

    print(f"✅ Bronze overwrite completed for: {table_name}")


In [0]:
ingest_to_bronze("branches", ";")
ingest_to_bronze("categories", ";")
ingest_to_bronze("customers", ";")
ingest_to_bronze("orders", ",")
ingest_to_bronze("order_details", ",")

✅ Bronze overwrite completed for: branches
✅ Bronze overwrite completed for: categories
✅ Bronze overwrite completed for: customers
✅ Bronze overwrite completed for: orders
✅ Bronze overwrite completed for: order_details
